# Maschinelles Lernen - Binäre Klassifikation

In diesem JN werden Fragestellungen der binären Klassifikation behandelt.
Eine binärer Klassifikation ist eine oftmals wiederkehrende Aufgabe:
- Ist in einem Bild ein Hund zu sehen oder nicht.
- Ist eine Person kreditwürdig oder nicht.
- Trennung bei physikalischen Experimenten zwischen Signal (*signal*) und Hintergrund (*background*)
- Ist ein Exoplant potenziell bewohnbar (*habitable*) oder nicht.

Die letzte Fragestellung wird hier etwas untersucht, so wie es auch bei Vivianana Acquaviva [Acq23] durchgeführt wird.
Dazu werden zuerst die Daten, die für die Untersuchung herangezogen werden betrachtet, um mit den Dtaen vertrauter zu werden.
Anschließend werden zwei ALgorithmen des ML vorgestellt: **Entscheidungsbaum** (*Decision Tree*, DT) und 
**k_Nächste-Nachbarn** (*k-Nearest Neighbour*, kNN) mit denen die Modelle trainiert werden.
Dann wird untersucht, wie wir das erzielte Ergebnis bewerten können, um damit zu untersuchen, welcher Algorithmus besser ist.
Da die Traingsdaten nur einen kleinen Umfang haben, wird das Verfahren **Cross Validation** vorgestellt, so dass 
die gesamte Lernmenge als Trainings- und Testdaten zur Verfügung steht.
Dabei werden die Modelle auf den gesamten Datenumfang, also nicht nur auf die Lernmenge angewendet.
Zum Abschluss werden die Ergebnisse ausgewertet und untersucht, welche Möglichkeiten zur Verbesserung es gibt.

Abschließend wird noch ein Ausblick auf die Multi Klassifizierung durchgeführt, denn für manche Fragestellung gibt es nicht eine binäre Entscheidung,
sondenr mehr als zwei Möglichkeiten:
- Irisblüten: Selektion zwischen drei Arten
- Erkenn von handschriftlichen Ziffern - Selektion zwischen zehn Ergebnissen.

## Problemstellung und Daten

Daten sind die Basis für die Arbeit im Data Science. Viviana Acquavia schreibt dazu (siehe [Acq23], Abschnitt 2.1) als von ihr aufgestellte *goldene Regel*: "know your data"!
Daher ist es wichtig, die Daten zu betrachten. Dabei sollen mindestens die nachfolgenden Dinge unternommen werden:
- Den Umfang der Daten prüfen, die Anzahl der Instanzen und die Anzahl der Eigenschaften. Gibt es ausreichend viele Datensätze für die Bearbeitung?
- Überprüfen, ob die Daten vollständig sind oder ob es in den Datensätzen fehlende Daten gibt. Wie wird mit mit fehlenden Daten umgegangen?
- Müssen die Daten eventuell normalisiert werden? Gibt es merkwürdige Verteilung der Daten? Wie wird mit Ausreißern (*outlier*) der Daten umgegangen?
- Sind bei einem Klassifizierungsproblem in allen Klassen ausreichend viele Daten vorhanden? 

Daher ist es nicht ratsam, die Algorithmene einfach nur anzuwenden, ohne dass die Daten verstanden werden. 
Die Ergebnisse der Analyse müssen geprüft und interpretiert werden. 
Es ist zu prüfen, ob das Ergebnis mit der Intuition übereinstimmt. Wenn nicht, dann ist zu klären, woher die Diskrepanz herrührt.

### Habitable Planeten

Vivana Acuavia wählt als ersten Beispiel einen Datensatz von 18 Exoplanten, bei denen verschiedene Merkmale bekannt sind 
und die Farge besteht, ob sie bewohnbar (*habitable*) sind. 
Die Daten hat sie von der Quelle http://phl.upr.edu/projects/habitable-exoplanets-catalog, in der inzwischen einige Tausende Exoplaneten aufgeführt sind. 
Sie hat 18 Planten aus dieser Liste ausgewählt. Diese Beispiele werden auch im nachfolgenden betrachtet und untersucht. 
Dabei werden verschiedene 
Modelle angewendet und dabei verschiedene Sachverhalte erläutert und beispielhaft demonstriert.

Die Beispieldaten beinhalten die folgenden Merkmale (*features*) und Ziele (*targets*):
- Name des Exoplanten: Dies ist nur für die Identifikation notwendig.
- Masse des Sterns (*stellar mass*), um den der Exoplant kreist, ausgedrückt in Sonnenmassen.
- Umlaufdauer (*orbital period*) des Exoplanten um den Stern in Tagen. 
- Distanz (distance) des Exoplanten von Stern in Astronomischen Einheiten (AU).
- Bewohnbar (*habitable*)? Eine Einschätzung, ob der Plant bewohnbar ist (1 = bewohnbar, 0 = nicht bewohnbar).

Hinweis: In der Quelle der Daten ist die Eigenschaft bewohnbar noch weiter differenziert. Darauf wird hier nicht genauer eingegangen. 
Wenn mit dem großen Datensatz aus der Quelle gearbeitet wird, dann ist zu klären, wie damit umgegangen wird. 
Daher ist es wichtig, die Daten zu kennen und vor den Auswertung zu betrachten.

In [1]:
# --- read the data
import pandas as pd
filename = 'HabitablePlanets_Examples18.csv'
data_planets = pd.read_csv (filename, sep=';')

# --- print the data of one exoplant
def print_planet (nr, planet):
    print (f"{nr:4d}  {planet [0]:20s} {planet [1]:6.2f}  {planet [2]:8.2f} {planet [3]:8.4f} {planet [4]:6d}")

# --- print a table with the complete lerarning set
def print_planets (plantes):
    print ("==============================================================")
    print ("  nr  name                   mass    period    dist.    hab.?")
    print ("                            (sun)     (day)     (AU)    (0=No)")
    print ("==============================================================")
    nr = 0
    for planet in plantes:
        print_planet (nr, planet)
        nr = nr + 1
    print ("==============================================================")
 
print_planets (data_planets.values)

  nr  name                   mass    period    dist.    hab.?
                            (sun)     (day)     (AU)    (0=No)
   0  Kepler-736 b           0.86      3.60   0.0437      0
   1  Kepler-636 b           0.85     16.08   0.1180      0
   2  Kepler-887 c           1.19      7.64   0.0804      0
   3  Kepler-442 b           0.61    112.30   0.4063      1
   4  Kepler-772 b           0.98     12.99   0.1074      0
   5  Teegarden's Star b     0.09      4.91   0.0252      1
   6  Ks-116 b               0.69      4.66   0.0481      0
   7  GJ 1061 c              0.12      6.69   0.0350      1
   8  HD 68402 b             1.12   1103.00   2.1810      0
   9  Kepler-1544 b          0.81    168.81   0.5571      1
  10  Kepler-296 e           0.50     34.14   0.1782      1
  11  Kepler-705 b           0.53     56.06   0.2319      1
  12  Kepler 445 c           0.18      4.87   0.0317      0
  13  HD 104067 b            0.62     55.81   0.2600      0
  14  GJ 4276 b              0.41  

Bei der Betrachtung der Daten fällt auf, dass der Plant mit der Nummer 8 (HD 68402 b) ein Ausreißer ist.
Die Periodendauer und der Abstand zur Sonne ist deutlicher größer als bei den anderen Planent.
In Anbetracht der Kepler'schen Planentengesetz ist das jedoch nicht unwahrscheinlich.

Es gibt bei den Beispielen acht bewohnbare Planeten und zehn unbewohnbare Planten. 
Damit ist der Datensatz bezüglich der Eigenschaft bewohnbar?" ausgeglichen.
Im großen Datensatz ist der Anteil der bewohnbaren Planten deutlich geringer.

Eine wichtige Eigenschaft für die Frage, ob ein Planet bewohnbar ist oder nicht, ist die Temperatur auf der Oberfläche.
Diese Eigenschaft ist nicht im Datensatz direkt enthalten. 
Über die Eigenschaften Sternenmasse und Distanz ist die Temperatur jedoch indirekt berücksichtigt, 
denn Sternenmasse und Distanz sind wichtige Parameter, welche die Temperatur stark beeinflussen.

### Aufspaltung in Traings- und Testdaten

Gemäß den Ausführungen bei Viviana Acquaviva werden die ersten 13 Planenten (also die Nummern 0 bis 12) als Trainingsdaten (*training set*) 
und die letzten fünf Planeten (Nummern 13 bis 17) als Testdaten (*test set*) verwendet. 

## Algorithmus: Entscheidungsbaum (*Decision Tree*, DT)

## Algorithmus: k-Nächste Nachbarn (*k-Nearest Neighbour*, kNN)

## Binäre Klassifikation - Bewertung

## Zwischenschritt: Bei wenigen Daten Cross Validation (CV)

## Auswertung der Ergebnisse

## Was kann verbessert werden

## Ausblick: Multi Klassifikation

## Literatur

- [Acq23] Viviana Acquaviva; Machine Learning for Physisics and Astronomy; Princeton University Press; 2023
